https://github.com/susanli2016/Machine-Learning-with-Python

## Always

In [ ]:
import csv
%load_ext autoreload
%autoreload 2
from progress import ProgressTracker

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle
import json

## LDA Preprocess

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

## Skip

In [ ]:
import random
text_data = []
with open('data/dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

In [ ]:
maxTweets = 4000000
tracker = ProgressTracker(maxTweets)
full_texts = []
twitter_accounts = []
with open('data/tweetsDFMin.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row, line_count in zip(csv_reader, range(maxTweets)):
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
        else:
            try:
                full_texts.append(row[1])
            except IndexError:
                continue
            try:
                twitter_accounts.append(row[2])
            except IndexError:
                full_texts.pop()
                pass
        tracker.update(line_count)
    print(f'Processed {line_count} lines.')
    print(f'Got {len(full_texts)} tweets')
    print(f'Got {len(twitter_accounts)} account ids')

In [ ]:
tweetsDF = pd.DataFrame({"full_text": full_texts, "twitter_account": twitter_accounts})

In [ ]:
tweetsDF.to_csv("data/tweetsDFMinReadable.csv", index=False)

## Get data

In [ ]:
tweetsDF = pd.read_csv("data/tweetsDFMinReadable.csv")
tweetsDF

In [ ]:
maxTweets = None
rawTweets = tweetsDF.loc[:maxTweets,"full_text"]

## Or, choosing to treat tweet sets as documents

In [ ]:
rawTweets = tweetsDF.groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))

In [ ]:
# preppedTweets = [prepare_text_for_lda(text) for text in rawTweets]

## Too slow

In [ ]:
test = rawTweets[0]
import json
json.loads(json.dumps(prepare_text_for_lda(text)))

In [ ]:
tweetsDF.loc[0, "tweet_word_roots"] = json.dumps(prepare_text_for_lda(text))

## Slightly faster if you initialize column first, not much

In [ ]:
tweetsDF["tweet_word_roots"] = np.nan

## Preprocess all

In [ ]:
tracker = ProgressTracker(len(rawTweets), 1)
preppedTweets = []
for i, text in enumerate(rawTweets):
    preppedTweets.append(prepare_text_for_lda(text))
    tracker.update(i)

In [ ]:
pickle.dump(preppedTweets, open("data/LDApreprocessedTweets.pkl", "wb"))

In [ ]:
preppedTweets1 = pickle.load(open("data/LDApreprocessedTweets.pkl", "rb"))
preppedTweets == preppedTweets1

In [ ]:
tweetsDF["preppedTweets"] = np.nan

In [ ]:
jsons = [json.dumps(preppedTweets[i]) if i < len(preppedTweets) else np.nan for i in range(len(tweetsDF))]

In [ ]:
tweetsDF["preppedTweets"] = jsons

In [ ]:
json.loads(tweetsDF.loc[0,"preppedTweets"])

In [ ]:
tweetsDF.to_csv("data/tweetsDFMinReadableLDApreprocessed.csv", index=False)

In [ ]:
tweetsDF = pd.read_csv("data/tweetsDFMinReadableLDApreprocessed.csv")

## Get groups in a good spot

In [ ]:
sum(preppedTweets[1:10]

In [ ]:
maxTweets = 500000
tweetsDF[:maxTweets].groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x)).reset_index()

In [ ]:
tweetsDF.preppedTweets[0]

In [ ]:
tweetsDF.columns

In [ ]:
tracker = ProgressTracker(2065, 5)
def decodeFlatten(df):
    print(df.columns)
    jsonlsts = df.preppedTweets
    print(jsonlsts[0])
    print(df.preppedTweets[0])
    raise RuntimeError
    tracker.update()
#     print(jsonlsts)
    lsts = [json.loads(arr) for arr in jsonlsts] 
    flatlst = []
    for lst in lsts:
        for item in lst:
            flatlst.append(item)
#     print("flatlist", flatlst)
    return json.dumps(lst)

In [ ]:
groupedDF = tweetsDF[:maxTweets].groupby('twitter_account').aggregate(decodeFlatten).reset_index()
groupedDF

In [ ]:
tweetsDF

In [ ]:
groupedDF.preppedTweets[0]

In [ ]:
groupedDF.to_csv("data/tweetsDFMinReadableLDApreprocessedGrouped.csv", index=False)
groupedDF = pd.read_csv("data/tweetsDFMinReadableLDApreprocessedGrouped.csv")

## test groupby

In [ ]:
df = pd.DataFrame({"GroupOnCol":[1,2,3,1,2,3], "ToGroupCol":["a","b","c","a","b","c"]})
df

In [ ]:
dfGoal = pd.DataFrame({"GroupOnCol":[1,2,3], "ToGroupCol":["aa","bb","cc"]})
dfGoal

In [ ]:
df.groupby("GroupOnCol").sum()

In [ ]:
df = pd.DataFrame({"GroupOnCol":[1,2,3,1,2,3], "ToGroupCol":[["a"],["b"],["c"],["a"],["b"],["c"]]})

In [ ]:
df

In [ ]:
df.groupby("GroupOnCol").sum()

In [ ]:
df = pd.DataFrame({"GroupOnCol":[1,2,3,1,2,3], "ToGroupCol":['["a"]','["b"]','["c"]','["a"]','["b"]','["c"]']})
df

In [ ]:
dfGoal = pd.DataFrame({"GroupOnCol":[1,2,3], "ToGroupCol":['["a","a"]', '["b","b"]', '["c","c"]']})
dfGoal

In [ ]:
def jsonSumLists(df):
    print(df)
#     raise RuntimeError
    return sum(df["ToGroupCol"])
df.groupby("GroupOnCol").aggregate(jsonSumLists)

In [ ]:
df["ToGroupCol"]

In [ ]:
index = pd.date_range('10/1/1999', periods=1100)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)
ts = ts.rolling(window=100, min_periods=100).mean().dropna()
ts.head()

In [ ]:
transformed = (ts.groupby(lambda x: x.year).apply(lambda x: (x - x.mean()) / x.std()))
transformed.head()

## Done preprocessing ^

# SCRAP

In [ ]:
import gensim

In [ ]:
from gensim import corpora

In [ ]:
import pyLDAvis.gensim

In [ ]:
tweetsDF = pd.read_csv("data/tweetsDFMinReadableLDApreprocessed.csv")

In [ ]:
arr = [json.loads(x) for x in tweetsDF["preppedTweets"]]

In [ ]:
sum(arr[:5])

In [ ]:
list(tweetsDF.loc[:4, "twitter_account"])

In [ ]:
df = pd.DataFrame({"names": list(tweetsDF.loc[:4,"twitter_account"]), "tweets": arr[:5]})

In [ ]:
df

In [ ]:
df.groupby("names").sum()

In [ ]:
tweetsDF["preppedTweets"] = arr

In [ ]:
tweetsDF[:10]

In [ ]:
len(tweetsDF)

In [ ]:
groupedDF = tweetsDF.loc[:, ("twitter_account", "preppedTweets")].groupby("twitter_account").sum().reset_index()

In [ ]:
groupedDF.to_csv("data/tweets/processed/GroupedLDA.csv", index=False)

In [ ]:
tweetsDF.loc[:, ("twitter_account", "preppedTweets")].to_csv("data/tweets/processed/TweetsLDA.csv", index=False)

In [ ]:
groupedDF = pd.DataFrame.from_csv("data/tweets/processed/TweetsLDA.csv")

In [ ]:
groupedDF = groupedDF.reset_index()

In [ ]:
groupedDF.reset_index().loc[0,"preppedTweets"]

In [ ]:
groupedDF[:5]

In [ ]:
groupedDF.loc[0,"preppedTweets"]

In [ ]:
groupedDF = pd.read_csv("data/tweetsDFMinReadableLDApreprocessedGrouped.csv")

In [ ]:
groupedDF

In [ ]:
open("hi1")

In [ ]:
pickle.load

In [ ]:
corpora.Dictionary.load

In [ ]:
import sys

In [ ]:
model = LDATwitterModel(tweetsDF=groupedDF, docNum=None, grouped=True)

In [ ]:
model.train()

In [ ]:
topics = model.ldamodel.print_topics(num_words=4)

In [ ]:
topics

In [ ]:
model.tweetsDF

In [ ]:
groupedDF

## Scrap

In [ ]:
text_data = preppedTweets

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
corpus[0:2] # document is tuples of word index, count. bag of words.

In [ ]:
dictionary.doc2bow(["hello", "hello"])

In [ ]:
import pickle
pickle.dump(corpus, open('data/corpus.pkl', 'wb'))
dictionary.save('data/dictionary.gensim')

### Try 20 topics

In [ ]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model20.gensim')

In [ ]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel.load("LDAModels/model20.gensim")

In [ ]:
i = 4
print(ldamodel.get_document_topics(corpus[i]))
print(text_data[i])

In [ ]:
tweetsDF.loc[i,"full_text"]

In [ ]:
for NUM_TOPICS in (5, 10):
    print(NUM_TOPICS)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(f'LDAModels/model{NUM_TOPICS}.gensim')

In [ ]:
maxTweets

In [ ]:
maxTweets = 100000
for doGroup in (False, True):
    print(f'Start doGroup={doGroup}')
    if doGroup:
        # 500 docs by long
        rawTweets = tweetsDF[:maxTweets*10].groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))
    else:
        # maxTweets docs by short
        rawTweets = tweetsDF.loc[:maxTweets,"full_text"]

    preppedTweets = []
    for i, text in enumerate(rawTweets):
        preppedTweets.append(prepare_text_for_lda(text))

    text_data = preppedTweets
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]

    for NUM_TOPICS in (5, 10, 20):
        if doGroup == False and NUM_TOPICS == 20:
            break
        print(doGroup, NUM_TOPICS)
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        ldamodel.save(f'LDAModels/model{NUM_TOPICS}.grouped={doGroup}.gensim')

In [ ]:
maxTweets = 100000
tweetsDF[:maxTweets*10].groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))

## Which model is best?

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel.load("LDAModels/model5.grouped=False.gensim")
i = 4
print(ldamodel.get_document_topics(corpus[i]))
print(text_data[i])

In [ ]:
ldamodel = LDATwitterModel(tweetsDF)

In [ ]:
len(corpus)

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

### pyLDAvis

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('data/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('LDAModels/model5.gensim')

In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

In [ ]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

In [ ]:
pickle.load(open("data/NLTKprocessedGroupedCorpus.pkl", "rb"))